# Predictive Analytics: Issue Priority Prediction (Breast Cancer Dataset)

This notebook demonstrates a complete workflow to **import**, **clean**, **label**, **split**, and **model** the Kaggle *Breast Cancer Wisconsin (Diagnostic)* dataset to predict **issue priority** (high / medium / low).

> ⚠️ Note: The original dataset has a **binary diagnosis** (Malignant vs Benign).  
> For this exercise, we derive a 3-class **priority** label using a simple, transparent rule: we first train a small risk model **only on the training set** to estimate malignancy probability; then we **bin** those probabilities into Low / Medium / High using the 33rd and 66th percentiles **computed on the training data**. We finally train a **Random Forest** to predict these priority labels on a held-out test set.


## 0. Environment & Reproducibility

- Uses Python, `pandas`, `scikit-learn`.  
- Random seeds fixed where applicable.  
- The notebook will try to download from **Kaggle** if your Kaggle API is configured; otherwise it will **fall back** to `sklearn.datasets.load_breast_cancer` (same UCI dataset).


In [2]:
# 1. Data Import (Kaggle if available, else sklearn fallback)

import os
import zipfile
import pandas as pd
from sklearn.datasets import load_breast_cancer

# Toggle this to force fallback without touching Kaggle:
FORCE_FALLBACK = False

def load_data():
    # If Kaggle credentials exist and not forcing fallback, try Kaggle
    kaggle_creds = os.path.expanduser("~/.kaggle/kaggle.json")
    if (not FORCE_FALLBACK) and os.path.exists(kaggle_creds):
        try:
            # This dataset is a mirror of the UCI Wisconsin (Diagnostic) data:
            # https://www.kaggle.com/uciml/breast-cancer-wisconsin-data
            print("Attempting Kaggle download...")
            os.system("kaggle datasets download -d uciml/breast-cancer-wisconsin-data -p ./data -q")
            os.makedirs("./data", exist_ok=True)
            # Unzip if needed
            for fname in os.listdir("./data"):
                if fname.endswith(".zip"):
                    with zipfile.ZipFile(os.path.join("./data", fname), "r") as zf:
                        zf.extractall("./data")
            # Look for CSV
            candidates = [f for f in os.listdir("./data") if f.lower().endswith(".csv")]
            if candidates:
                csv_path = os.path.join("./data", candidates[0])
                df = pd.read_csv(csv_path)
                print(f"Loaded Kaggle CSV: {csv_path}")
                return df
        except Exception as e:
            print("Kaggle download failed, falling back to sklearn:", e)

    # Fallback to sklearn (UCI) dataset (same content, different packaging)
    print("Falling back to sklearn.load_breast_cancer")
    sk = load_breast_cancer(as_frame=True)
    df = sk.frame.copy()
    df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
    # Align naming to Kaggle where possible:
    # Kaggle has 'diagnosis' as 'M'/'B'
    df['diagnosis'] = df['target'].map({0: 'M', 1: 'B'})
    return df

df = load_data()
df.head()


Falling back to sklearn.load_breast_cancer


,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0,M
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0,M
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0,M
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0,M
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0,M


## 2. Clean & Label

- **Cleaning**: standardize column names, drop obvious identifiers if present (e.g., `id`), check nulls.  
- **Labeling**: derive **priority** ∈ {`low`, `medium`, `high`} from a *risk proxy* computed *only on the training set*:
  1. Split into **train/test** first (to avoid leakage).
  2. On **train only**, fit a simple logistic regression (with CV) to obtain malignancy probabilities.
  3. Compute **33rd** and **66th** percentile thresholds on these train probabilities.
  4. Map probability → priority: `low` (≤ p33), `medium` (p33–p66], `high` (> p66).
  5. Apply the **same thresholds** to test-set probabilities produced by the train-fitted model.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Basic cleaning
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
df = df.copy()

# Drop an 'id' column if present in Kaggle CSV
if 'id' in df.columns:
    df = df.drop(columns=['id'])

# Ensure 'diagnosis' column exists (Kaggle has 'diagnosis' B/M)
if 'diagnosis' not in df.columns and 'target' in df.columns:
    df['diagnosis'] = df['target'].map({0:'M', 1:'B'})

# Split before deriving priority
features = [c for c in df.columns if c not in ('diagnosis','target')]
X = df[features]
y_diag = df['diagnosis']  # 'M' or 'B'

X_train, X_test, y_train_diag, y_test_diag = train_test_split(
    X, y_diag, test_size=0.2, random_state=42, stratify=y_diag
)

# Risk proxy model (train only)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logit = LogisticRegression(max_iter=500)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Out-of-fold probs on TRAIN
# cross_val_predict with method='predict_proba' can sometimes return a 2D array
# (n_samples, n_classes). Be defensive: extract the column corresponding to
# the positive class (here class '1' meaning diagnosis == 'M').
oof_raw = cross_val_predict(
    logit, X_train_scaled, (y_train_diag == 'M').astype(int),
    cv=cv, method='predict_proba'
)

if oof_raw.ndim == 1:
    oof_proba = oof_raw.astype(float)
else:
    # If there are two columns, take the probability for class 1 (positive)
    # otherwise take the single column available.
    oof_proba = oof_raw[:, 1] if oof_raw.shape[1] > 1 else oof_raw[:, 0]

# Fit on all TRAIN, then score TEST
logit.fit(X_train_scaled, (y_train_diag == 'M').astype(int))
test_raw = logit.predict_proba(X_test_scaled)

if test_raw.ndim == 1:
    test_proba = test_raw.astype(float)
else:
    test_proba = test_raw[:, 1] if test_raw.shape[1] > 1 else test_raw[:, 0]

# Compute thresholds on TRAIN (use nan-aware quantile just in case)
p33 = np.nanpercentile(oof_proba, 33.333)
p66 = np.nanpercentile(oof_proba, 66.666)

def to_priority(p):
    if p <= p33:
        return 'low'
    elif p <= p66:
        return 'medium'
    else:
        return 'high'

y_train_pri = pd.Series([to_priority(p) for p in oof_proba], index=X_train.index)
y_test_pri = pd.Series([to_priority(p) for p in test_proba], index=X_test.index)

# Quick sanity check — print counts so the cell output is clear and won't silently create a tuple
print(pd.Series(y_train_pri).value_counts(normalize=True).rename('train_class_mix'))
print(pd.Series(y_test_pri).value_counts(normalize=True).rename('test_class_mix'))


(high      0.334066
 low       0.334066
 medium    0.331868
 Name: train_class_mix, dtype: float64,
 low       0.385965
 medium    0.324561
 high      0.289474
 Name: test_class_mix, dtype: float64)

## 3. Model: Random Forest

We fit a **RandomForestClassifier** to predict the 3-class priority on the **training** split and evaluate on the **test** split.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

rf = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    class_weight='balanced_subsample'
)
rf.fit(X_train_scaled, y_train_pri)\n
y_pred = rf.predict(X_test_scaled)\n
acc = accuracy_score(y_test_pri, y_pred)
f1_macro = f1_score(y_test_pri, y_pred, average='macro')

print("Accuracy:", acc)
print("Macro F1:", f1_macro)
print("\nClassification report:\n", classification_report(y_test_pri, y_pred, digits=4))


Accuracy: 0.9035087719298246
Macro F1: 0.9057095475005923

Classification report:
               precision    recall  f1-score   support

        high     0.9412    0.9697    0.9552        33
         low     0.9500    0.8636    0.9048        44
      medium     0.8250    0.8919    0.8571        37

    accuracy                         0.9035       114
   macro avg     0.9054    0.9084    0.9057       114
weighted avg     0.9069    0.9035    0.9039       114



## 4. Results (Accuracy & F1)

Below are the key results captured from a test run:

**Key Results (on test set)**  
- Accuracy: **0.9035**  
- Macro F1-score: **0.9057**

<details>
<summary>Classification report</summary>

```
              precision    recall  f1-score   support

        high     0.9412    0.9697    0.9552        33
         low     0.9500    0.8636    0.9048        44
      medium     0.8250    0.8919    0.8571        37

    accuracy                         0.9035       114
   macro avg     0.9054    0.9084    0.9057       114
weighted avg     0.9069    0.9035    0.9039       114

```
</details>


## 5. Notes & Next Steps

- The 3-class **priority** is a derived target for demonstration purposes (the source data is binary).  
- Consider **calibrated** models and domain-approved thresholds for real triage systems.  
- Try alternative models (XGBoost, LightGBM) and perform **hyperparameter tuning**.  
- Add **confusion matrix** and **feature importance** plots for more insight.
